# Study of $q_0$ 

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
# needed imports

Using the likelihood 
$$\mathcal{L}(y|\mu+b,\vec{\chi}) = \frac{e^{-(\mu+b)}(\mu+b)^y}{y!}\frac{e^{-\frac{(\mu-\mu_0)^2}{2\sigma_\mu^2}}}{\sqrt{2\pi\sigma_\mu^2}}\frac{e^{-\frac{(b-b_0)^2}{2\sigma_b^2}}}{\sqrt{2\pi\sigma_b^2}}$$
Which gives (keeping only "interesting terms")
$$\ell(\mu,b) = -(\mu+b)+y\ln(\mu+b) - \frac{1}{2}\left(\frac{\mu-\mu_0}{\sigma_\mu}\right)^2 - \frac{1}{2} \left(\frac{b-b_0}{\sigma_b}\right)^2$$
Studying the case for $\sigma_s\rightarrow\infty$ the third term above becomes zero. From this we can find the MLEs for two cases.

First, finding $\hat\mu$, which is done by setting $\frac{\partial\ell(\mu,b)}{\partial\mu}=0$, this gives us $\hat\mu = y - b_0$. Furthermore we can find $\hat b$ by setting $\frac{\partial\ell(\hat\mu,b)}{\partial b}=0$, giving $\hat b = b_0$

Then for the second case, the null hypothesis we find $\hat{\hat b}$ by setting $\frac{\partial\ell(0,b)}{\partial b}=0$, this yields $\hat{\hat{b}} =\frac{b_0-\sigma_b^2+\sqrt{(b_0-\sigma_b^2)^2+4y\sigma_b^2}}{2}$ 

Using these values for the likelihood ratio test statistics gives us

$$
q_0 = -2[\ell(s=0,\hat{\hat b}) - \ell(\hat s, \hat b)]
$$
Giving 
$$
q_0 = 2(y\ln\frac{y}{\hat{\hat b}} + \hat{\hat b} - y) + \left(\frac{\hat{\hat b} -b_0}{\sigma_b}\right)^2
$$

In [2]:
def log_likelihood(mu, b, y, b_0, db):
    ell = -(mu+b) + y*np.log(mu+b) -0.5*((b-b_0)/db)**2
    return ell

def test_Stat_null(y, b_hhat, b_0, db):
    q_0 = 2*(y*np.log(y/b_hhat)+b_hhat - y) + ((b_hhat-b_0)/db)**2
    return q_0

Using the asympotics approximation of Brazzale et al we start by defining the likelihood root

$$
r(\mu) = sign(\hat\mu-\mu)\sqrt{2(\ell_p(\hat\mu, \hat b) - \ell_p(\mu, \hat{\hat b}))} 
$$
where $\ell_p(\mu) = \ell(\mu,\hat{\hat b})$ is the profile log likelihood. For our purposes we (should) have that
$$
r(0) = \sqrt{q_0}
$$

In [3]:
def likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
    ell_p_hat = log_likelihood(mu_hat, b_hat, y, b_0, db)
    ell_p = log_likelihood(mu, b_hhat, y, b_0, db)
    r = np.sign(mu_hat- mu)*np.sqrt(2*(ell_p_hat-ell_p))
    return r

Furthermore we have the modification of the likelihood root (for examples with the exponential family, so not this! This is a bad approximation!)
$$
r^*(\mu) = r(\mu) + \frac{1}{r(\mu)}\log\left(\frac{q(\mu)}{r(\mu)}\right)
$$
where
$$
q(\mu) = t(\mu) = \sqrt{j_p(\hat\mu)} (\hat\mu - \mu)
$$
is the Wald statistic, or
$$
q(\mu) = s(\mu) = \sqrt{j_p(\hat\mu)}^{-1}\frac{\partial\ell_p(\mu)}{\partial\mu}
$$
is the score statistic. And where 
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2}
$$
is the observed information function. 

For our purposes, we have that 
$$
\frac{\partial\ell_p(\mu)}{\partial\mu} = -1 + \frac{y}{\mu + \hat{\hat b}}
$$
and
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2} = \frac{y}{(\mu + \hat{\hat b})^2}
$$

In [4]:
def info_func(mu, b_hatt, y):
    j = y/(mu+b_hatt)**2
    return j

def wald(mu_hat, mu, b_hatt, y):
    j = info_func(mu_hat, b_hatt, y)
    t = np.sqrt(j)*(mu_hat-mu)
    return t

def score(mu_hat, mu, b_hatt, y):
    j = info_func(mu_hat, b_hatt, y)
    s = np.sqrt(1/j)*( -1 + y/(mu+b_hatt))
    return s

def modified_likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db, doWald=True):
    r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
    if doWald == True:
        q_mu = wald(mu_hat, mu, b_hhat, y)
    else: 
        q_mu = score(mu_hat, mu, b_hhat, y)
    return r_mu + (1 / r_mu) * np.log(q_mu / r_mu)


Now for an example using $y=3$, $b_0=0.78$ and $\sigma_b=0.18$, first we compare that indeed $r(0)=q_0$

In [ ]:
y_obs = 3
b_0 = 0.78
sigma_b = 0.18

mu_hat = y_obs - b_0
b_hatt = (b_0-sigma_b**2+np.sqrt((b_0-sigma_b**2)**2+4*y_obs*(sigma_b**2)))/2
b_hat = b_0

r = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

q_0 = test_Stat_null(y_obs, b_hatt, b_0, sigma_b)

print(r)
print(np.sqrt(q_0))

1.8477368150191664
1.8477368150191662


Now we check all the modifications

In [ ]:
r_s = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b) # Wald
r_ss = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b, doWald=False) # Score
r_mp = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b) # Mid-p
r_s_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b)
r_ss_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b, doWald=False)


r_sf = stats.norm.sf(r, loc=0)
r_star_sf = stats.norm.sf(r_s, loc=0)
r_stars_sf = stats.norm.sf(r_ss, loc=0)
r_sf_mp = stats.norm.sf(r_mp, loc=0)
r_star_sf_mp = stats.norm.sf(r_s_mp, loc=0)
r_stars_sf_mp = stats.norm.sf(r_ss_mp, loc=0)



In [ ]:
#Making a table of the results...

data_sf = {'r': [r_sf, r], 'r* Wald': [r_star_sf, r_s], 'r* score': [r_stars_sf, r_ss], 
        'Mid-P r': [r_sf_mp, r_mp], 'Mid-P r* Wald': [r_star_sf_mp, r_s_mp], 'Mid-P r* score': [r_stars_sf_mp, r_ss_mp]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 


,r,r* Wald,r* score,Mid-P r,Mid-P r* Wald,Mid-P r* score
p-value,0.032320,0.050978,0.010171,0.015410,0.026114,0.005342
Significance,1.847737,1.635446,2.319975,2.159381,1.941244,2.552854
